In [1]:
!pip install requests

In [6]:
from bs4 import BeautifulSoup
import requests

In [7]:
result=requests.get('https://www.google.com/')

In [8]:
result.status_code

200

In [14]:
result.headers

{'Date': 'Sun, 04 Dec 2022 16:52:51 GMT', 'Expires': '-1', 'Cache-Control': 'private, max-age=0', 'Content-Type': 'text/html; charset=ISO-8859-1', 'Cross-Origin-Opener-Policy-Report-Only': 'same-origin-allow-popups; report-to="gws"', 'Report-To': '{"group":"gws","max_age":2592000,"endpoints":[{"url":"https://csp.withgoogle.com/csp/report-to/gws/other"}]}', 'P3P': 'CP="This is not a P3P policy! See g.co/p3phelp for more info."', 'Content-Encoding': 'gzip', 'Server': 'gws', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'Set-Cookie': 'SOCS=CAAaBgiAxq-cBg; expires=Wed, 03-Jan-2024 16:52:51 GMT; path=/; domain=.google.com; Secure; SameSite=lax, AEC=AakniGNq4pzXtu9hKd-3sx-O3whPniUgmPVReNdmQaj7gQHqr68fvLJzfxE; expires=Fri, 02-Jun-2023 16:52:51 GMT; path=/; domain=.google.com; Secure; HttpOnly; SameSite=lax, __Secure-ENID=8.SE=43GAfHHm-iijcea1LplUjWfBhPyPHUuDB6pIKQgORbbeyUgV7OCEZN1_7Xy_I-pF9SHsiXMxv3yV6FxfqIaqaHG4S1FGuh_Jij2xVgi1HhQvnRR9JaAESUJRJ7bPwYB5lf72Se4lLvxand82-MJP_f13yxdKD

In [20]:
src=result.content

In [16]:
print(src)

b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en-IE"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/logos/doodles/2022/seasonal-holidays-2022-6753651837109831.2-law.gif" itemprop="image"><meta content="Seasonal Holidays 2022" property="twitter:title"><meta content="" property="twitter:description"><meta content="summary_large_image" property="twitter:card"><meta content="@GoogleDoodles" property="twitter:site"><meta content="https://www.google.com/logos/doodles/2022/seasonal-holidays-2022-6753651837109831.2-2xa.gif" property="twitter:image"><meta content="https://www.google.com/logos/doodles/2022/seasonal-holidays-2022-6753651837109831.2-2xa.gif" property="og:image"><meta content="1150" property="og:image:width"><meta content="460" property="og:image:height"><meta content="https://www.google.com/logos/doodles/2022/seasonal-holidays-2022-6753651837109831.2-2xa.gif" property="og:url"><meta content="video.other" pr

In [28]:
soup= BeautifulSoup(src)

In [29]:
links=soup.find_all('a')

In [30]:
for link in links:
       if 'About' in link.text:
            print(link)
            print(link.attrs['href'])

<a href="/intl/en/about.html">About Google</a>
/intl/en/about.html


In [42]:
src1=requests.get('https://www.whitehouse.gov/briefing-room/')

In [43]:
content=src1.content

In [44]:
soup1=BeautifulSoup(content)

In [75]:
x=[]
for h2_tag in soup1.find_all('h2'):
   

None
